# Create general scripts to process entire dataset

*Andrea Soto*  
*MIDS W205 Final Project*  
*Project Name: Graph Model of the Million Song Dataset*

---

# Step 1: Create a list of HDF5 and JSON files

In [6]:
%%writefile scripts/list_MDS_files.py
#!/usr/bin/env python
import os
import glob
import sys
import shutil

def main(inDir, outDir, overwrite = False):
    
    try:
        os.path.exists(inDir)
    except:
        print "Input file: '%s' does not exist"%(inDir)
    else:
        outFile = outDir + '/list_hdf5_files.txt'
        if not os.path.exists(outFile) or overwrite:
            # List all paths of songs
            get_song_paths = glob.glob(inDir+'/*/*/*/*.h5')
            
            if not get_song_paths:
                print "No HDF5 (.h5) files foung in '%s'"%(inDir)
                print "Check that the file structure under '%s' is /*/*/*/song_files.h5"%(inDir)
            else:
                with open(outFile,'w') as f:
                    f.writelines('\n'.join(p for p in get_song_paths))
                    f.close()
                print "File '%s' successfully created"%(outFile)
        else:
            print "File '%s' already exists"%(outFile)
    
if __name__ == '__main__':
    '''
    Creates the file 'list_hdf5_files.txt' with the list of HDF5 files
    
    USE:
    python list_MDS_files.py <path to songs> <save list path> <OPTIONAL overwrite>
    
    Paths should NOT include '/' at the end
    If the file already exists, it will not be overwritten. Send 'True' to overwrite
    '''
    
    input_path = sys.argv[1]  
    output_path = sys.argv[2]
    
    # Option to overwrite current file
    overwrite = False
    if len(sys.argv) > 3:
        overwrite  = sys.argv[3]
    
    main(input_path, output_path, overwrite)

Overwriting scripts/list_MDS_files.py


In [7]:
%%writefile scripts/list_LastFM_files.py
#!/usr/bin/env python
import os
import glob
import sys
import shutil

def main(inDir, outDir, overwrite = False):
    
    try:
        os.path.exists(inDir)
    except:
        print "Input file: '%s' does not exist"%(inDir)
    else:
        outFile = outDir + '/list_lastfm_files.txt'
        if not os.path.exists(outFile) or overwrite:
            # List all paths of songs
            get_song_paths = glob.glob(inDir+'/*/*/*/*.json')
            
            if not get_song_paths:
                print "No JSON files foung in '%s'"%(inDir)
                print "Check that the file structure under '%s' is /*/*/*/song_files.json"%(inDir)
            else:
                with open(outFile,'w') as f:
                    f.writelines('\n'.join(p for p in get_song_paths))
                    f.close()
                print  "File '%s' successfully created"%(outFile)
        else:
            print "File '%s' already exists"%(outFile)
    
if __name__ == '__main__':
    '''
    Creates the file 'list_lastfm_files.txt' with the list of HDF5 files
    
    USE:
    python list_MDS_files.py <path to songs> <save list path> <OPTIONAL overwrite>
    
    Paths should NOT include '/' at the end
    If the file already exists, it will not be overwritten. Send 'True' to overwrite
    '''
    input_path = sys.argv[1]  
    output_path = sys.argv[2]
    
    # Option to overwrite current file
    overwrite = False
    if len(sys.argv) > 3:
        overwrite  = sys.argv[3]
    
    main(input_path, output_path, overwrite)

Overwriting scripts/list_LastFM_files.py


In [40]:
!python scripts/list_MDS_files.py MillionSongSubset/data test

File 'test/list_hdf5_files.txt' successfully created


In [56]:
!python scripts/list_LastFM_files.py MillionSongSubset/lastfm_subset test

File 'test/list_lastfm_files.txt' successfully created


In [59]:
!ls -l test

total 1068
-rw-rw-r-- 1 asoto asoto 509999 Dec 14 07:01 list_hdf5_files.txt
-rw-rw-r-- 1 asoto asoto 578459 Dec 14 07:13 list_lastfm_files.txt


# Step 2: Load and Transform Data in Spark

In [8]:
%%writefile scripts/extractData.py
#!/usr/bin/env python
import os
import glob
import sys
import shutil
from pyspark import SparkContext
import numpy as np
import h5py
import json

def main(inDir, outDir, mismatchFile):
    
    # === Start Spark Context ===
    sc = SparkContext(appName="SparkProcessing")
    
    # =====================================================================
    # === Load mismatches ===
    toRemoveRDD = sc.textFile('file://'+mismatchFile+'/sid_mismatches.txt').map(parse_mismatches)
    songsToRemove = sc.broadcast(toRemoveRDD.collect())
    
    # === Load list of files ===
    song_pathsRDD   = sc.textFile('file://' + inDir + '/list_hdf5_files.txt')
    lastfm_pathsRDD = sc.textFile('file://' + inDir + '/list_lastfm_files.txt')
    
    # === Extract Song Data ===
    songsRDD = song_pathsRDD.map(get_h5_info).cache()
    lastfmRDD = lastfm_pathsRDD.map(get_json_info).cache()
    
    # =====================================================================
    # == Delete Sub-Folders ===
    folders = ['/nodes_artists','/nodes_songs','/nodes_albums','/nodes_years','/nodes_tags',
              '/rel_similar_artists', '/rel_performs','/rel_artist_has_album','/rel_artist_has_tag',
               '/rel_song_in_album','/rel_similar_songs', '/rel_song_has_tag', '/rel_song_year']
    for p in folders:
        if os.path.exists(outDir+p):
            shutil.rmtree(outDir+p)
    
    # =====================================================================
    # === Create Nodes ===
    
    # === ARTISTS ===
    # CSV Format: artist_id, artist_mb_id, artist_7d_id, artist_name
    fields = ['artist_id', 'artist_mbid', 'artist_7did', 'artist_name']
    fieldsBrC = sc.broadcast(fields)
    songsRDD.map(makeCSVline).distinct().saveAsTextFile('file://'+outDir+'/nodes_artists')
    
    # === SONGS ===
    # CSV Format: song_id, track_id, song_title, danceability, duration, energy, loudness
    fields = ['song_id', 'track_id', 'title', 'dance', 'dur', 'energy','loudness']
    fieldsBrC = sc.broadcast(fields)
    songsRDD.map(makeCSVline).distinct().saveAsTextFile('file://'+outDir+'/nodes_songs')
    
    # === ALBUMS ===
    # CSV Format: album_name
    songsRDD.map(lambda x: x['album']).distinct().saveAsTextFile('file://'+outDir+'/nodes_albums')
    
    # === YEAR ===
    # CSV Format: year
    songsRDD.map(lambda x: x['year']).filter(
        lambda x: int(x) > 0).distinct().saveAsTextFile('file://'+outDir+'/nodes_years')
    
    # === TAGS ===
    # CSV Format: tag_name
    artistTags = songsRDD.flatMap(lambda x: x['a_terms']).distinct()
    songTags = lastfmRDD.flatMap(lambda x: x['tags']).map(lambda x: x[0]).distinct()
    allTags = songTags.union(artistTags).distinct()
    allTags.saveAsTextFile('file://'+outDir+'/nodes_tags')
    
    # =====================================================================
    # === Create Relationships ===
    
    # === SIMILAR_TO relationship between artist and artist ===
    # CSV Format: from_artist_id, to_artist_id
    # Similar Artist to Artist (directional, no properties)
    similarArtistsRDD = songsRDD.map(lambda x: (x['artist_id'],x['a_similar'])).flatMapValues(lambda x: x)
    similarArtistsRDD.distinct().map(lambda x: x[0]+","+x[1]).saveAsTextFile('file://'+outDir+'/rel_similar_artists')
    
    # === PERFORMS relationship between artist and song ===
    # CSV Format: artist_id, song_id
    # Artist Performs Song (directional, no properties)
    songsRDD.map(lambda x: x['artist_id']+","+x['song_id']).distinct().saveAsTextFile('file://'+outDir+'/rel_performs')
    
    # === HAS_ALBUM relationship between artist and album  ===
    # CSV Format: artist_id, album_name
    # Artist Has Album (directional, no properties)
    songsRDD.map(lambda x: x['artist_id']+","+x['album']).distinct().saveAsTextFile('file://'+outDir+'/rel_artist_has_album')

    # === HAS_TAG relationship between artist and tags ===
    # CSV Format: artist_id, tag_name, tag_frequency, tag_weight
    # Artist Has Tags (directional, has properties frequency and weight)
    songsRDD.flatMap(artistToTags).distinct().saveAsTextFile('file://'+outDir+'/rel_artist_has_tag')
    
    # === IN_ALBUM relationship between song and album ===
    # CSV Format: song_id, album_name
    # Song In Album (direction, no properties)   
    songsRDD.map(lambda x: x['song_id']+","+x['album']).distinct().saveAsTextFile('file://'+outDir+'/rel_song_in_album')
    
    # === SIMILAR_TO relationship between song and song ===
    #CSV Format: from_track_id, to_track_id, similarity_measure
    # Similar Song to Song (directional, with property similarity measure)   
    similarSongsRDD = lastfmRDD.filter(
        lambda x: x['similars']<>[]).map(
        lambda x: (x['track_id'],x['similars'])).flatMapValues(lambda x: x)
    similarSongsRDD.map(
        lambda x: x[0]+","+x[1][0]+","+str(x[1][1])).saveAsTextFile('file://'+outDir+'/rel_similar_songs')
    
    # === HAS_TAG relationship between song and tags
    # CSV Format: track_id, tag_name, tag_weight
    lastfmRDD.flatMap(songToTags).saveAsTextFile('file://'+outDir+'/rel_song_has_tag')
    
    # === RELEASED_ON relationship between song and year
    # CSV Format: song_id, year
    # Song Released in Year (directional, no properties)
    songsRDD.filter(lambda x: int(x['year'])<>0).map(
        lambda x: x['song_id']+","+str(x['year'])).saveAsTextFile('file://'+outDir+'/rel_song_year')


    # =====================================================================
    # === Stop Spark Context ===
    sc.stop()


def parse_mismatches(line):
    '''
    This function extracts the songID and trackID of the mismatched records.
    Returned value: ('songID', 'trackID')
    '''
    return line[8:45].split()


def get_h5_info(path):
    '''
    Takes a path to a song stored as an HDF5 file and returns a dictionary with the 
    information that will be included in the graph
    ''' 
    d = {}
    with h5py.File(path, 'r') as f:
        song_id = f['metadata']['songs']['song_id'][0]
        track_id = f['analysis']['songs']['track_id'][0]
        
        if (song_id, track_id) not in songsToRemove.value:

            # --- Artist Info -----------------------------
            d.setdefault('artist_id', f['metadata']['songs']['artist_id'][0])
            d.setdefault('artist_mbid', f['metadata']['songs']['artist_mbid'][0])
            d.setdefault('artist_7did', f['metadata']['songs']['artist_7digitalid'][0])
            d.setdefault('artist_name', f['metadata']['songs']['artist_name'][0])

            # --- Song Info -----------------------------
            d.setdefault('song_id', song_id)
            d.setdefault('track_id', track_id)
            d.setdefault('title', f['metadata']['songs']['title'][0])
            d.setdefault('dance', f['analysis']['songs']['danceability'][0])
            d.setdefault('dur', f['analysis']['songs']['duration'][0])
            d.setdefault('energy', f['analysis']['songs']['energy'][0])
            d.setdefault('loudness', f['analysis']['songs']['loudness'][0])

            # --- Year -----------------------------
            d.setdefault('year', f['musicbrainz']['songs']['year'][0])

            # --- Album -----------------------------
            d.setdefault('album', f['metadata']['songs']['release'][0])

            # --- Similar Artist -----------------------------
            d.setdefault('a_similar', np.array(f['metadata']['similar_artists']))

            # --- Artist Terms -----------------------------
            d.setdefault('a_terms', np.array(f['metadata']['artist_terms']))
            d.setdefault('a_tfrq', np.array(f['metadata']['artist_terms_freq']))
            d.setdefault('a_tw', np.array(f['metadata']['artist_terms_weight']))

            return d
        else: 
            pass

def get_json_info(path):
    with open(path) as data_file:    
        return json.load(data_file)

def makeCSVline(line):
    return ','.join(str(line[field]) for field in fieldsBrC.value)
    
def artistToTags(record):
    '''
    Concatenate artist with each tag
    Normalize tag frequency and weight
    '''
    normalize_frq = record['a_tfrq'] / sum(record['a_tfrq'])
    normalize_w = record['a_tw'] / sum(record['a_tw'])
    terms = record['a_terms']
    artist = record['artist_id']
    
    result = []
    for i in range(len(terms)):
        result.append( artist +","+ terms[i] +","+ str(normalize_frq[i]) +","+ str(normalize_w[i]))
    
    return result

def songToTags(record):
    '''
    Concatenate song with each tag
    '''
    tags = record['tags']
    total_weight = sum(float(w[1]) for w in tags)
    track_id = record['track_id']
    
    result = []
    for i in range(len(tags)):
        result.append( track_id +","+ tags[i][0] +","+ str(float(tags[i][1])/total_weight))
    
    return result

if __name__ == '__main__':
    '''
    input_path: path to where the list of hdf5 and json files was created
    output_path: a temporary directory where the Spark CSV files separated as part-000xx files will be stored
    mismatch_path: path to where the mismatches file is located
    
    DO NOT INCLUDE '/' AT THE END OF PATH
    Cannot change file names
    '''
    input_path = sys.argv[1]  
    output_path = sys.argv[2]
    mismatch_path = sys.argv[2]
    
    main(input_path, output_path, mismatch_path)

Overwriting scripts/extractData.py


---

---

# General script structure with spark context

In [ ]:
%%writefile test_code/count_h5.py
#!/usr/bin/env python
from pyspark import SparkContext
import time
import h5py

def read_h5_file(path):
    with h5py.File(path, 'r') as f:
        return f['metadata']['songs']['title'][0]
#Start Time
t1 = time.time()

# --- Process files ----
sc = SparkContext(appName="SparkHDF5")
file_paths = sc.textFile('file:///data/asoto/projectW205/data/list_files.txt')

songs = file_paths.map(read_h5_file)
songs.count()
# ----------------------

#End Time
t2 = time.time()
sec = t2-t1

print "Run Time: %0.2f sec = %.2f min = %.2f h"%(sec,sec/60.0,sec/1440.0)
sc.stop()

In [ ]:
!spark-submit test_code/count_h5.py